## MNSIT Conditional GenAI
Using Numpy Only


#### Candidate
Muhammad Mehroz
<br>

I could not succeeded to create the model using simple numpy


In [14]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
np.random.seed(10)
random_dim = 100
img_shape = (28, 28)
latent_dim = 100
num_classes = 10
epochs = 15005
batch_size = 128
save_interval = 1000

# Load MNIST dataset
def load_mnist():
    from tensorflow.keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = (x_train.astype('float32') - 127.5) / 127.5, (x_test.astype('float32') - 127.5) / 127.5
    x_train = x_train.reshape((-1, *img_shape, 1))
    x_test = x_test.reshape((-1, *img_shape, 1))
    return x_train, y_train, x_test, y_test

# Generator
def generate_noise(batch_size):
    return np.random.normal(0, 1, (batch_size, latent_dim))

def generate_labels(batch_size):
    return np.random.randint(0, num_classes, batch_size)

def define_generator():
    model = dict(
        dense1=np.random.randn(latent_dim, 128 * 7 * 7) * 0.1,
        batch_norm1=np.ones((128 * 7 * 7,)),
        batch_norm2=np.ones((128,)),
        conv2d1=np.random.randn(3, 3, 128, 128) * 0.1,
        batch_norm3=np.ones((64,)),
        conv2d2=np.random.randn(3, 3, 64, 1) * 0.1
    )
    return model



def batch_norm(x, mean, variance, epsilon=1e-5):
    # Reshape mean and variance to match the shape of x along the batch dimension
    mean_broadcasted = np.broadcast_to(mean, x.shape)
    variance_broadcasted = np.broadcast_to(variance, x.shape)
    
    # Apply batch normalization
    normalized_x = (x - mean_broadcasted) / np.sqrt(variance_broadcasted + epsilon)
    return normalized_x


def generate_images(generator, noise, labels):
    z = np.dot(noise, generator['dense1'])
    z_mean = np.mean(z, axis=0, keepdims=True)
    z_var = np.var(z, axis=0, keepdims=True)
    z = z.reshape((-1, 128, 7, 7))
    
    # Batch normalization for z
    z = batch_norm(z, z_mean, z_var)
    
    z = np.maximum(0, z)
    z = np.repeat(z[:, np.newaxis, :, :], 2, axis=1)
    z = np.repeat(z[:, :, :, np.newaxis], 2, axis=3)
    
    z = np.dot(z, generator['conv2d1'])
    z_mean = np.mean(z, axis=(0, 1, 2), keepdims=True)
    z_var = np.var(z, axis=(0, 1, 2), keepdims=True)
    
    # Batch normalization for z
    z = batch_norm(z, z_mean, z_var)
    
    z = np.maximum(0, z)
    z = np.dot(z, generator['conv2d2'])
    z_mean = np.mean(z, axis=(0, 1, 2), keepdims=True)
    z_var = np.var(z, axis=(0, 1, 2), keepdims=True)
    
    # Batch normalization for z
    z = batch_norm(z, z_mean, z_var)
    
    z = np.tanh(z)
    return z

# Discriminator
def define_discriminator():
    model = dict(
        conv2d1=np.random.randn(3, 3, 2, 32) * 0.1,
        conv2d2=np.random.randn(3, 3, 32, 64) * 0.1,
        dense1=np.random.randn(7 * 7 * 64, 512) * 0.1,
        dense2=np.random.randn(512, 1) * 0.1
    )
    return model

def discriminator_predict(discriminator, images, labels):
    z = np.concatenate((images, labels), axis=3)
    z = np.dot(z, discriminator['conv2d1'])
    z = np.maximum(0.2 * z, z)
    z = np.dot(z, discriminator['conv2d2'])
    z = np.maximum(0.2 * z, z)
    z = z.reshape((z.shape[0], -1))
    z = np.dot(z, discriminator['dense1'])
    z = np.maximum(0.2 * z, z)
    z = np.dot(z, discriminator['dense2'])
    z = 1 / (1 + np.exp(-z))
    return z

# Training
def train(generator, discriminator, x_train, y_train):
    losses = {'G': [], 'D': []}
    for epoch in range(epochs):
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_imgs, labels = x_train[idx], y_train[idx]

        noise = generate_noise(batch_size)
        fake_imgs = generate_images(generator, noise, labels)

        d_loss_real = np.mean(-np.log(discriminator_predict(discriminator, real_imgs, labels) + 1e-9))
        d_loss_fake = np.mean(-np.log(1 - discriminator_predict(discriminator, fake_imgs, labels) + 1e-9))
        d_loss = 0.5 * (d_loss_real + d_loss_fake)

        z = generate_noise(batch_size)
        labels = generate_labels(batch_size)
        g_loss = np.mean(-np.log(discriminator_predict(discriminator, generate_images(generator, z, labels), labels) + 1e-9))

        losses['G'].append(g_loss)
        losses['D'].append(d_loss)

        if epoch % save_interval == 0:
            print(f"Epoch {epoch}, Losses (D, G): {d_loss}, {g_loss}")
            results = generate_images(generator, z_test, labels_test)
            plot_image(results[:16], labels_test[:16], 4, 4)
    return losses

# Plotting
def plot_image(images, labels, rows, cols):
    fig = plt.figure(figsize=(8, 8))
    for i in range(1, cols * rows + 1):
        img = images[i - 1]
        ax = fig.add_subplot(rows, cols, i)
        ax.title.set_text(labels[i - 1])
        plt.imshow(img.reshape(img_shape), cmap='gray')
        plt.axis('off')
    fig.tight_layout()
    plt.show()

# Main
x_train, y_train, x_test, y_test = load_mnist()
z_test = generate_noise(16)
labels_test = generate_labels(16)

generator = define_generator()
discriminator = define_discriminator()
losses = train(generator, discriminator, x_train, y_train)



ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (1,6272)  and requested shape (128,128,7,7)